In [1]:
import dash
import more_itertools
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import matplotlib.pyplot as plt
# Load the data using pandas
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')


In [3]:
app=dash.Dash(__name__)

df = df[(df['Year'] >= 1980) & (df['Year'] <= 2013)]
app.layout=html.Div([
    html.H1("Automobile Sales Statistics Dashboard",style={'textAlign':'center','color':'#503D3','fontsize':24}),
    html.Div([
        html.Div([
            html.Label('Select Report Type'),
            dcc.Dropdown(
                id='dropdown-statistics',
                options=[ {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},{'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}],
                value='Select Statistics',
                placeholder='Select a report type'
            )
        ],style={ 'width': '48%','display': 'inline-block'}),
       

    html.Div([
        html.Label('Select Year for Yearly Statistics'),
        dcc.Dropdown(
            id='select-year',
            options=[{'label': str(year), 'value': year} for year in sorted(df['Year'].unique())],
            value='Select-Year',
            placeholder='Select-Year'
        )
    ],style={'width': '48%', 'display': 'inline-block', 'float': 'right'})
    ]),
    html.Div([
            html.Div(id='output-container', className='chart-list'),])

    ])

@app.callback(Output(component_id='select-year', component_property='disabled'),
              Input(component_id='dropdown-statistics', component_property='value')
             )


def update_dropdown(report_type):
    return report_type=='Recession Period Statistics'

@app.callback(Output(component_id='output-container', component_property='children'),
             [Input(component_id='dropdown-statistics', component_property='value'),
             Input(component_id='select-year', component_property='value')]
             )
def update_ouput_container(report_type,input_year):
    if report_type=='Recession Period Statistics':
        rec_data=df[df['Recession']==1]
        #figure 1
        rec_avg=rec_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        fig1=dcc.Graph(figure=px.line(rec_avg,x='Year',y='Automobile_Sales',title='Sales During Recession Periods'))

        #figure 2
        rec_vehicle=rec_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        fig2=dcc.Graph(figure=px.bar(rec_vehicle,x='Vehicle_Type',y='Automobile_Sales',title='Average Sales by Vehicle Type (Recessions)'))

        #figure 3
        rec_pie=rec_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        fig3=dcc.Graph(figure=px.pie(rec_pie,values='Advertising_Expenditure',names='Vehicle_Type',title='Advertising Expenditure During Recessions'))

        #figure 4
        unemp_data=rec_data.groupby('Vehicle_Type').agg({
            'unemployment_rate': 'mean','Automobile_Sales': 'mean'
        }).reset_index()
        fig4=dcc.Graph(figure=px.bar(unemp_data,x='Vehicle_Type',y='Automobile_Sales',color='unemployment_rate',color_continuous_scale='RdYlGn_r',labels={'unemployment_rate': 'Unemployment Rate (%)','Automobile_Sales': 'Average Sales'},title='Effect of Unemployment Rate on Vehicle Sales During Recessions'))

        return [ html.Div([fig1, fig2], style={'display': 'flex'}),html.Div([fig3, fig4], style={'display': 'flex'}) ]
    else:
        y_data=df[df['Year']==input_year]
        #fig1
        lin_data=df.groupby('Year')['Automobile_Sales'].mean().reset_index()
        fig5=dcc.Graph(figure=px.line(lin_data,x='Year',y='Automobile_Sales',title='Yearly Average Automobile Sales Trend'))
        #fig 2
        data_month=y_data.groupby('Month')['Automobile_Sales'].sum().reset_index()
        fig6=dcc.Graph(figure=px.line(data_month,x='Month',y='Automobile_Sales',title=f'Monthly Automobile Sales in {input_year}'))
        #fig3
        data_v=y_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        fig7=dcc.Graph(figure=px.bar(data_v,x='Vehicle_Type',y='Automobile_Sales',title=f'Average Sales by Vehicle Type in {input_year}'))

        #fig 4
        data_p=y_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        fig8=dcc.Graph(figure=px.pie(data_p,values='Advertising_Expenditure',names='Vehicle_Type',title=f'Advertising Expenditure in {input_year}'))
        return [ html.Div([fig5,fig6],style={'display':'flex'}),html.Div([fig7,fig8],style={'display':'flex'})]

if __name__=='__main__':
    app.run(debug=True,port=1236)
        
    